***GENERATED CODE FOR customer PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3332', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phone', 'transformation_label': 'String Indexer'}], 'feature': 'Phone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': '327-1058', 'max': '422-9964', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phone'}, {'feature_label': 'Phone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'Intl_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intl_Plan'}, {'feature_label': 'Intl_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intl_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'}], 'feature': 'VMail_Plan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VMail_Plan'}, {'feature_label': 'VMail_Plan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VMail_Plan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'}], 'feature': 'cluster_labels', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3332', 'mean': '', 'stddev': '', 'min': 'day_callers', 'max': 'vmailers', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'cluster_labels'}, {'feature_label': 'cluster_labels', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('cluster_labels')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
#%run customerHooks.ipynb
try:
	#sourcePreExecutionHook()

	predictivechurntrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/TrainData/PredictiveChurnTrain.csv', 'filename': 'PredictiveChurnTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(predictivechurntrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run customerHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(predictivechurntrain,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "AK", "max": "WY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Account_Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "101.06", "stddev": "39.83", "min": "1", "max": "243", "missing": "0"}, "updatedLabel": "Account_Length"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Area_Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "437.19", "stddev": "42.38", "min": "408", "max": "510", "missing": "0"}, "updatedLabel": "Area_Code"}, {"transformationsData": [{"feature_label": "Phone", "transformation_label": "String Indexer"}], "feature": "Phone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "327-1058", "max": "422-9964", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Intl_Plan", "transformation_label": "String Indexer"}], "feature": "Intl_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intl_Plan"}, {"transformationsData": [{"feature_label": "VMail_Plan", "transformation_label": "String Indexer"}], "feature": "VMail_Plan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VMail_Plan"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "VMail_Message", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "8.09", "stddev": "13.69", "min": "0", "max": "51", "missing": "0"}, "updatedLabel": "VMail_Message"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "179.75", "stddev": "54.46", "min": "0.0", "max": "350.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "100.43", "stddev": "20.07", "min": "0", "max": "165", "missing": "0"}, "updatedLabel": "Day_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Day_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "30.56", "stddev": "9.26", "min": "0.0", "max": "59.64", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Day_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "200.98", "stddev": "50.72", "min": "0.0", "max": "363.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "100.11", "stddev": "19.93", "min": "0", "max": "170", "missing": "0"}, "updatedLabel": "Eve_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Eve_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "17.08", "stddev": "4.31", "min": "0.0", "max": "30.91", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Eve_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "200.86", "stddev": "50.58", "min": "23.2", "max": "395.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "100.11", "stddev": "19.57", "min": "33", "max": "175", "missing": "0"}, "updatedLabel": "Night_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Night_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "9.04", "stddev": "2.28", "min": "1.04", "max": "17.77", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Night_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "10.24", "stddev": "2.79", "min": "0.0", "max": "20.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_Mins", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "591.83", "stddev": "89.94", "min": "284.3", "max": "885.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_Mins"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "4.48", "stddev": "2.46", "min": "0", "max": "20", "missing": "0"}, "updatedLabel": "Intl_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intl_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "2.76", "stddev": "0.75", "min": "0.0", "max": "5.4", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intl_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Charge", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3332", "mean": "59.44", "stddev": "10.5", "min": "22.93", "max": "96.15", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total_Charge"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CustServ_Calls", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "1.56", "stddev": "1.32", "min": "0", "max": "9", "missing": "0"}, "updatedLabel": "CustServ_Calls"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3332", "mean": "0.14", "stddev": "0.35", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Churn"}, {"transformationsData": [{"feature_label": "cluster_labels", "transformation_label": "String Indexer"}], "feature": "cluster_labels", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3332", "mean": "", "stddev": "", "min": "day_callers", "max": "vmailers", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "cluster_labels"}, {"feature": "State_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3332", "mean": "22.54", "stddev": "14.55", "min": "0.0", "max": "50.0", "missing": "0"}, "updatedLabel": "State_stringindexer"}, {"feature": "Phone_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3332", "mean": "1665.5", "stddev": "962.01", "min": "0.0", "max": "3331.0", "missing": "0"}, "updatedLabel": "Phone_stringindexer"}, {"feature": "Intl_Plan_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3332", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Intl_Plan_stringindexer"}, {"feature": "VMail_Plan_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "3332", "mean": "0.28", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "VMail_Plan_stringindexer"}, {"feature": "cluster_labels_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "3332", "mean": "2.39", "stddev": "1.7", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "cluster_labels_stringinde..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run customerHooks.ipynb
try:
	#sinkPreExecutionHook()

	newproject = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/60c1c8234dbd4f59f72e000a/part-00000-90990d67-86e4-4f5b-8d44-6aa5a7761797-c000.csv', 'samplecount': 319, 'originalcount': 3332, 'orignalKey': '/FileStore/platform/TrainData/PredictiveChurnTrain.csv', 'pathOnly': '/new project', 'project_id': '60c04f1c71c23f37322815cb', 'parent_id': '60c04f1c71c23f37322815cb', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e006e', 'field': 'State_stringindexer', 'alias': 'State_stringindexer', 'type': 'string'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e006f', 'field': 'Account_Length', 'alias': 'Account_Length', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0070', 'field': 'Area_Code', 'alias': 'Area_Code', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0071', 'field': 'Phone_stringindexer', 'alias': 'Phone_stringindexer', 'type': 'string'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0072', 'field': 'Intl_Plan_stringindexer', 'alias': 'Intl_Plan_stringindexer', 'type': 'string'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0073', 'field': 'VMail_Plan_stringindexer', 'alias': 'VMail_Plan_stringindexer', 'type': 'string'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0074', 'field': 'VMail_Message', 'alias': 'VMail_Message', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0075', 'field': 'Day_Mins', 'alias': 'Day_Mins', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0076', 'field': 'Day_Calls', 'alias': 'Day_Calls', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0077', 'field': 'Day_Charge', 'alias': 'Day_Charge', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0078', 'field': 'Eve_Mins', 'alias': 'Eve_Mins', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0079', 'field': 'Eve_Calls', 'alias': 'Eve_Calls', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007a', 'field': 'Eve_Charge', 'alias': 'Eve_Charge', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007b', 'field': 'Night_Mins', 'alias': 'Night_Mins', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007c', 'field': 'Night_Calls', 'alias': 'Night_Calls', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007d', 'field': 'Night_Charge', 'alias': 'Night_Charge', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007e', 'field': 'Intl_Mins', 'alias': 'Intl_Mins', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e007f', 'field': 'total_Mins', 'alias': 'total_Mins', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0080', 'field': 'Intl_Calls', 'alias': 'Intl_Calls', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0081', 'field': 'Intl_Charge', 'alias': 'Intl_Charge', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0082', 'field': 'Total_Charge', 'alias': 'Total_Charge', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0083', 'field': 'CustServ_Calls', 'alias': 'CustServ_Calls', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0084', 'field': 'Churn', 'alias': 'Churn', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0085', 'field': 'cluster_labels_stringindexer', 'alias': 'cluster_labels_stringindexer', 'type': 'string'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0086', 'field': 'State_stringindexer', 'alias': 'State_stringindexer', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0087', 'field': 'Phone_stringindexer', 'alias': 'Phone_stringindexer', 'type': 'real'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0088', 'field': 'Intl_Plan_stringindexer', 'alias': 'Intl_Plan_stringindexer', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e0089', 'field': 'VMail_Plan_stringindexer', 'alias': 'VMail_Plan_stringindexer', 'type': 'numeric'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89c4dbd4f59f72e008a', 'field': 'cluster_labels_stringindexer', 'alias': 'cluster_labels_stringindexer', 'type': 'real'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e008b', 'field': 'State', 'alias': 'State', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e008c', 'field': 'Account_Length', 'alias': 'Account_Length', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e008d', 'field': 'Area_Code', 'alias': 'Area_Code', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e008e', 'field': 'Phone', 'alias': 'Phone', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e008f', 'field': 'Intl_Plan', 'alias': 'Intl_Plan', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0090', 'field': 'VMail_Plan', 'alias': 'VMail_Plan', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0091', 'field': 'VMail_Message', 'alias': 'VMail_Message', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0092', 'field': 'Day_Mins', 'alias': 'Day_Mins', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0093', 'field': 'Day_Calls', 'alias': 'Day_Calls', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0094', 'field': 'Day_Charge', 'alias': 'Day_Charge', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0095', 'field': 'Eve_Mins', 'alias': 'Eve_Mins', 'type': 'real', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0096', 'field': 'Eve_Calls', 'alias': 'Eve_Calls', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0097', 'field': 'Eve_Charge', 'alias': 'Eve_Charge', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0098', 'field': 'Night_Mins', 'alias': 'Night_Mins', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e0099', 'field': 'Night_Calls', 'alias': 'Night_Calls', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009a', 'field': 'Night_Charge', 'alias': 'Night_Charge', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009b', 'field': 'Intl_Mins', 'alias': 'Intl_Mins', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009c', 'field': 'total_Mins', 'alias': 'total_Mins', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009d', 'field': 'Intl_Calls', 'alias': 'Intl_Calls', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009e', 'field': 'Intl_Charge', 'alias': 'Intl_Charge', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e009f', 'field': 'Total_Charge', 'alias': 'Total_Charge', 'type': 'real', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e00a0', 'field': 'CustServ_Calls', 'alias': 'CustServ_Calls', 'type': 'numeric', 'position': '21'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e00a1', 'field': 'Churn', 'alias': 'Churn', 'type': 'numeric', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '60c1c89d4dbd4f59f72e00a2', 'field': 'cluster_labels', 'alias': 'cluster_labels', 'type': 'String', 'position': '23'}], 'server': 'http://13.59.224.229:3200', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'ggg.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-6c2a759c-272f-4219-9a26-bb12486cd2f5-c000.csv', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
